In [1]:
#from games_setup import *
import init
from SBMLLint.common import constants as cn
from SBMLLint.common.molecule import Molecule, MoleculeStoichiometry
from SBMLLint.common.reaction import Reaction
from SBMLLint.games.som import SOM
from SBMLLint.common.simple_sbml import SimpleSBML

import collections
import itertools
import networkx as nx
import numpy as np
import os
import pandas as pd
#
from SBMLLint.common.stoichiometry_matrix import StoichiometryMatrix
from SBMLLint.games.mesgraph import MESGraph
from SBMLLint.games.games_pp import GAMES_PP, SOMStoichiometry, SOMReaction, TOLERANCE
from SBMLLint.games.games_report import GAMESReport, SimplifiedReaction
NULL_STR = ""
ReactionOperation = collections.namedtuple("ReactionOperation", 
    "reaction operation")
STATPHOSPHORYLATION = "statPhosphorylation"
PSTATDIMERISATION = "PstatDimerisation"
PSTATDIMERISATIONNUC = "PstatDimerisationNuc"
SPECIES_TEST = "species_test"
PSTAT_SOL = "Pstat_sol"
PSTATDIMER_NUC = "PstatDimer_nuc"
PSTAT_NUC = "Pstat_nuc"
STAT_SOL = "stat_sol"
PSTATDIMER_SOL = "PstatDimer_sol"

In [2]:
PREFIX = "BIOMD0000000"

def load_file_from_curated_data(num):

  format_num = format(num, '03d')
  filename = "%s%s_url.xml" % (PREFIX, format_num)
  path = os.path.join(cn.BIOMODELS_DIR, filename)
  simple = SimpleSBML()
  simple.initialize(path)
  return simple

In [3]:
simple = load_file_from_curated_data(167)
m = GAMES_PP(simple)
m.analyze()
gr = GAMESReport(m)

We just analyzed the data...
Type I error:  []
Type II error:  []
Canceling error:  []
Echelon error:  [statPhosphorylation:  -> {species_test}]
Type III error:  [PstatDimerisation: 2.00 {Pstat_sol} -> 2.00 {Pstat_nuc=stat_nuc=stat_sol}]


In [4]:
report, error_num = gr.reportEchelonError(m.echelon_errors, explain_details=True)
print(report[-288:])

will result in empty reactant with zero mass:

:  -> {species_test}


----------------------------------------------------------------------

----------------------------------------------------------------------



**********************************************************************




In [14]:
extended_report = NULL_STR
extended_report = extended_report + "will result in empty reactant with zero mass:\n\n:  -> {species_test}\n\n"
extended_report = extended_report + "\n----------------------------------------------------------------------\n"
extended_report = extended_report + "\n----------------------------------------------------------------------\n\n"
extended_report = extended_report + "\n\n**********************************************************************\n\n"
print(report[-288:] == extended_report)

True


In [4]:
report, error_num = gr.reportTypeThreeError(m.type_three_errors, explain_details=True)
print(report)

We detected a mass imbalance from the following reactions:

1. PstatDimerisation: 2.00 Pstat_sol -> PstatDimer_sol
2. PstatDimerisationNuc: 2.00 Pstat_nuc -> PstatDimer_nuc

----------------------------------------------------------------------
----------------------------------------------------------------------

These uni-uni reactions created mass-equivalence.
(The chemical species within a curly bracket have the same atomic mass.)

{Pstat_nuc=stat_nuc=stat_sol} is inferred by:
3. stat_export: stat_sol -> stat_nuc
4. statDephosphorylation: Pstat_nuc -> stat_nuc

{PstatDimer_nuc=PstatDimer_sol} is inferred by:
5. PstatDimer__import: PstatDimer_sol -> PstatDimer_nuc

----------------------------------------------------------------------

These multi-uni reactions created mass-inequality.

6. statPhosphorylation: stat_sol -> Pstat_sol + species_test

----------------------------------------------------------------------

Based on the reactions above, we have mass-equivalent pseudo rea

In [6]:
report

'We detected a mass imbalance from the following reactions:\n\n1. PstatDimerisation: 2.00 Pstat_sol -> PstatDimer_sol\n2. PstatDimerisationNuc: 2.00 Pstat_nuc -> PstatDimer_nucWe detected a mass imbalance from the following reactions:\n\n1. PstatDimerisation: 2.00 Pstat_sol -> PstatDimer_sol\n2. PstatDimerisationNuc: 2.00 Pstat_nuc -> PstatDimer_nuc\n\n----------------------------------------------------------------------\n----------------------------------------------------------------------\n\nThese uni-uni reactions created mass-equivalence.\n(The chemical species within a curly bracket have the same atomic mass.)\n\n{Pstat_nuc=stat_nuc=stat_sol} is inferred by:\n3. stat_export: stat_sol -> stat_nuc\n4. statDephosphorylation: Pstat_nuc -> stat_nuc\n\n{PstatDimer_nuc=PstatDimer_sol} is inferred by:\n5. PstatDimer__import: PstatDimer_sol -> PstatDimer_nuc\n\n\n----------------------------------------------------------------------\n\nThese multi-uni (uni-multi) reactions created mass-i